# **Final Project**

Class: GPU28EE

Author: Larissa Wierzynski Kulik


> ***Summary:*** The idea of this project is to use a GPU to process colored images, applying the Gaussian Filter and the Convole method. For development, I will use the lib Cupy, which will help me program in CUDA using Python.

In [11]:
!conda create -n cuda84 python=3.11 -y

/bin/bash: line 1: conda: command not found


In [12]:
from numba import cuda
from math import exp
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import time

In [13]:
from numba import cuda
print(cuda.runtime.get_version())

(12, 5)


In [14]:
@cuda.jit
def convolve(result, mask, image):
    i, j = cuda.grid(2)
    image_rows, image_cols = image.shape
    if(i >= image_rows) or (j >= image_cols):
        return

    delta_row = mask.shape[0] // 2
    delta_col = mask.shape[1] // 2

    s = 0
    for k in range(mask.shape[0]):
        for l in range(mask.shape[1]):
            i_k = i - k + delta_row
            j_l = j - l + delta_col
            if (i_k >= 0) and (i_k < image_rows) and (j_l >= 0) and (j_l < image_cols):
                s += image[i_k, j_l] * mask[k, l]
    result[i,j] = s

In [15]:
@cuda.jit
def gaussian_gpu(sigma, kernel_size, kernel):
  m = kernel_size // 2
  n = kernel_size // 2

  x = cuda.threadIdx.x
  y = cuda.threadIdx.y

  kernel[x,y] = exp(-((x-m)**2 + (y-n)**2) / (2*sigma**2))

In [16]:
sigma = 0.0001
kernel_size = 30

kernel = np.zeros((kernel_size, kernel_size), np.float32)
d_kernel = cuda.to_device(kernel)

In [17]:
image = np.asarray(ImageOps.grayscale(Image.open('images.jpg'))) # aqui
d_image = cuda.to_device(image)

In [18]:
d_result = cuda.device_array_like(image)

In [19]:
gaussian_gpu[(1,), (kernel_size, kernel_size)](sigma, kernel_size, d_kernel)

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
ERROR:numba.cuda.cudadrv.driver:Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.5; current version is '8.4'

In [ ]:
blockdim = (32,32)
griddim = (image.shape[0] // blockdim[0] + 1, image.shape[1] // blockdim[1] + 1)

In [ ]:
convolve[griddim, blockdim](d_result, d_kernel, d_image)
result = d_result.copy_to_host()

In [ ]:
plt.figure()
plt.imshow(image, cmap='gray')
plt.title('Original image')
plt.show()

In [ ]:
plt.figure()
plt.imshow(result, cmap='gray')
plt.title('Filtered image')
plt.show()